In [1]:
import pandas as pd
import numpy as np
import warnings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.options.display.max_colwidth = 1000
warnings.filterwarnings("ignore")

In [2]:
class GetDummies():
    def __init__(self,  part = 0.01):
        self.part = part
        self.unique_values = {}
        self.cat_columns = []
        self.num_columns = []

    def fit(self, df):
        self.cat_columns = list(df.columns[df.dtypes == 'object'])
        self.num_columns = list(df.columns[df.dtypes == 'int']) + list(df.columns[df.dtypes == 'float'])
        for i in self.cat_columns:
            tmp = df[i].value_counts(normalize=True).reset_index()
            tmp = tmp[tmp[i] >= self.part]
            self.unique_values[i] = list(tmp['index'].unique())

    def transform(self, df, inplace = False):
        if inplace:
            columns = []
            for i in self.cat_columns:
                for j in range(len(self.unique_values[i])):
                    columns.append(str(i + '=' + str(self.unique_values[i][j])))
                    if i in df.columns:
                        df[i + '=' + str(self.unique_values[i][j])] = \
                        df[i].apply(lambda x: 1 if x == str(self.unique_values[i][j]) else 0)
            return df
        else:
            df_ = df.copy()
            columns = []
            for i in self.cat_columns:
                for j in range(len(self.unique_values[i])):
                    columns.append(str(i + '=' + str(self.unique_values[i][j])))
                    if i in df_.columns:
                        df_[i + '=' + str(self.unique_values[i][j])] = \
                        df_[i].apply(lambda x: 1 if x == str(self.unique_values[i][j]) else 0)
            return df_

In [3]:
class TargetPercentageEncoding():
    def __init__(self):
        self.fit_dict = {}
        self.cat_columns = []
    def fit(self, df, target_column, part = 0.05):
        self.cat_columns = list(df.columns[df.dtypes == 'object'])
        
        for i in self.cat_columns:
            tmp = df[[i,target_column]].groupby(i)[target_column].mean().reset_index()
            tmp_ = df.groupby(i).size().reset_index().rename(columns={i:i, 0 : 'CNT'})
            tmp_['CNT'] = tmp_['CNT'] / tmp_['CNT'].sum()
            tmp = tmp.merge(tmp_)
            tmp = tmp[tmp['CNT'] >= part]
            self.fit_dict[i] = pd.Series(tmp[target_column].values,index = tmp[i]).to_dict()
    
    def transform(self, df, inplace = False):
        if inplace:
            for i in self.cat_columns:
                df[i+'_TARGET_PERCENTAGE'] = df[i].map(self.fit_dict[i]).fillna(0)
            return df
        else:
            df_ = df.copy()
            for i in self.cat_columns:
                df_[i+'_TARGET_PERCENTAGE'] = df_[i].map(self.fit_dict[i]).fillna(0)
            return df_

In [4]:
class ValueCountEncoding():
    def __init__(self):
        self.fit_dict = {}
        self.cat_columns = []
        
    def fit(self, df):
        self.cat_columns = list(df.columns[df.dtypes == 'object'])
        for i in self.cat_columns:
            tmp = df.groupby(i).size().reset_index().rename(columns={i:i, 0 : 'CNT'})
            tmp['CNT'] = tmp['CNT'] / tmp['CNT'].sum()
            self.fit_dict[i] = pd.Series(tmp['CNT'].values,index = tmp[i]).to_dict()
    
    def transform(self, df, inplace = False):
        if inplace:
            for i in self.cat_columns:
                df[i+'_FREQUENCY'] = df[i].map(self.fit_dict[i]).fillna(0)
            return df
        else:
            df_ = df.copy()
            for i in self.cat_columns:
                df_[i+'_FREQUENCY'] = df_[i].map(self.fit_dict[i]).fillna(0)
            return df_

In [5]:
a = GetDummies()
a.fit(train)
a.transform(train)
# b = GetDummies()
# b.fit(train)
# a.transform(test).merge(b.transform(test))

NameError: name 'train' is not defined

In [319]:
train = pd.DataFrame([['Sayan',float('Nan'),'Astana',190, 90,1],
                        ['Batima','F','Kostanay',162,54,0],
                        ['Nadira','F','Astana',165,50.001,0],
                        ['Zhanbolat','M','Aqtobe',179,70,1],
                        ['Azamat','M','Aqtobe',179,75,1]
                       ],
                       columns = ['NAME', 'SEX', 'CITY','HEIGHT', 'WEIGHT', 'TARGET'])
test = pd.DataFrame([['Madiyar','SEX','Kostanay',185, 72,''],
                        ['Marat','sd','Pavlodar',180,float('Nan'),''],
                        ['Dair','sd','Karaganda',184,80.011,''],
                        ['Madina','sd','Astana',
                         float('Nan')
                         ,52,''],
                        ['Perizat','sd','Pavlodar',165,50,'']
                       ],
                       columns = ['NAME','SEX', 'CITY','HEIGHT', 'WEIGHT', 'TARGET'])

In [3]:
# class DataProcessing():
#     def __init__(self, train, test):
#         self.train = train
#         self.test = test
    
#     def delete_columns_in_train():
        

#     def categorial_columns(self):
#         return self.train.columns[self.train.dtypes == 'object']

#     def numeric_columns(self):
#         categorial_columns = self.categorial_columns()
#         numeric_columns = list(set(self.train.columns) - set(categorial_columns))
#         for col in numeric_columns:
#             if len(self.train[col].unique()) <= 2:
#                 numeric_columns.remove(col)
#         return numeric_columns

#     def categorial_column_unique_values(self, part=0.01):
#         categorial_column_unique_values = {}
#         for i in self.categorial_columns():
#             categorial_column_unique_values[i] = (self.train[i].value_counts(normalize=True) \
#                                                       [self.train[i].value_counts(normalize=True) > part].index)
#         return categorial_column_unique_values

#     def fillna(self, nan='Median', column_index=False):
#         #         Работает
#         train_ = train.copy()
#         test_ = test.copy()

#         def max_frequency(lst):
#             return max(set(lst), key=lst.count)

#         if nan == 'Median':
#             for column in self.numeric_columns():
#                 train_[column].fillna(train_[column].median(), inplace=True)
#                 test_[column].fillna(train_[column].median(), inplace=True)

#             for column in self.categorial_columns():
#                 train_[column].fillna(max_frequency(list(train_[column])), inplace=True)
#                 test_[column].fillna(max_frequency(list(train_[column])), inplace=True)

#         elif nan == 'Null':
#             for column in self.numeric_columns():
#                 train_[column].fillna(0, inplace=True)
#                 test_[column].fillna(0, inplace=True)

#             for column in self.categorial_columns():
#                 train_[column].fillna('No_data', inplace=True)
#                 test_[column].fillna('No_data', inplace=True)
#         elif nan == 'Mean':
#             for column in self.numeric_columns():
#                 train_[column].fillna(train_[column].mean(), inplace=True)
#                 test_[column].fillna(train_[column].mean(), inplace=True)

#             for column in self.categorial_columns():
#                 train_[column].fillna(max_frequency(list(train_[column])), inplace=True)
#                 test_[column].fillna(max_frequency(list(train_[column])), inplace=True)
#         else:
#             return 'Invaild parametr'
#         train_['TRAIN_TEST'] = 'TRAIN'
#         test_['TRAIN_TEST'] = 'TEST'
#         return train_.append(test_)

#     def coding_by_column(self, categorial_columns, numeric_columns, func='mean'):
#         if func in ['mean', 'max', 'min', 'sum']:
#             train_ = train.copy()
#             test_ = test.copy()

#             columns = []

#             for i in categorial_columns:
#                 for j in numeric_columns:
#                     train_[i].fillna(0, inplace=True)
#                     tmp = train_.groupby(i).agg({j: func}).reset_index().rename(columns={i: i,
#                                                                                          j: i + '_' + j + '_' + func.upper()
#                                                                                          })
#                     train_ = train_.merge(tmp)
#                     test_[i].fillna(0, inplace=True)
#                     tmp = test_.groupby(i).agg({j: func}).reset_index().rename(columns={i: i,
#                                                                                         j: i + '_' + j + '_' + func.upper()
#                                                                                         })

#                     columns.append(i + '_' + j + '_' + func.upper())
#                     test_ = test_.merge(tmp)

#             return train_.append(test_)[columns]
#         else:
#             return 'Argument func has invalid value'

#     def add_nan_index(self):
#         #         Работает
#         train_ = train.copy()
#         test_ = test.copy()
#         columns = train_.columns[train_.isna().any()].tolist()
#         for i in columns:
#             train_[i + '=NAN'] = list(train_['CITY'].isna().astype(int))
#             test_[i + '=NAN'] = list(test_['CITY'].isna().astype(int))
#         columns = [i + '=NAN' for i in columns]
#         return train_.append(test_)[columns]

#     def get_data(self):
#         #         Работает
#         train_ = train.copy()
#         test_ = test.copy()
#         train_['TRAIN_TEST'] = 'TRAIN'
#         test_['TRAIN_TEST'] = 'TEST'
#         return train_.append(test_)

#     def value_counts(self):
#         #         Работает
#         train_ = self.train.copy()
#         test_ = self.test.copy()
#         train_.fillna('NO_DATA', inplace=True)
#         test_.fillna('NO_DATA', inplace=True)
#         columns = []
#         for i in self.categorial_columns():
#             tmp = train_[i].value_counts(normalize=True). \
#                 reset_index(). \
#                 rename(columns={'index': i, i: i + '_PART'})
#             train_ = train_.merge(tmp, how='left')
#             columns.append(i + '_PART')
#         for i in self.categorial_columns():
#             tmp = train_[i].value_counts(normalize=True). \
#                 reset_index(). \
#                 rename(columns={'index': i, i: i + '_PART'})
#             test_ = test_.merge(tmp, how='left')
#             df = train_.append(test_)[columns].fillna(0, inplace=False)
#         return df

#     def value_counts_by_target(self):
#         #         Работает
#         train_ = train.copy()
#         test_ = test.copy()
#         for i in self.categorial_columns():
#             tmp = train_.query('TARGET==1')[i].fillna('NO_DATA').value_counts(normalize=True). \
#                 reset_index(). \
#                 rename(columns={'index': i, i: i + '_PART_TARGET=1'})
#             train_ = train_.fillna('NO_DATA', inplace=False).merge(tmp, how='left')
#             train_[i + '_PART_TARGET=1'].fillna(0, inplace=True)

#         for i in self.categorial_columns():
#             tmp = train_.query('TARGET==0')[i].fillna('NO_DATA').value_counts(normalize=True). \
#                 reset_index(). \
#                 rename(columns={'index': i, i: i + '_PART_TARGET=0'})
#             train_ = train_.fillna('NO_DATA', inplace=False).merge(tmp, how='left')
#             train_[i + '_PART_TARGET=0'].fillna(0, inplace=True)

#         for i in self.categorial_columns():
#             tmp = train_.query('TARGET==1')[i].fillna('NO_DATA').value_counts(normalize=True). \
#                 reset_index(). \
#                 rename(columns={'index': i, i: i + '_PART_TARGET=1'})
#             test_ = test_.fillna('NO_DATA', inplace=False).merge(tmp, how='left')
#             test_[i + '_PART_TARGET=1'].fillna(0, inplace=True)

#         for i in self.categorial_columns():
#             tmp = train_.query('TARGET==0')[i].fillna('NO_DATA').value_counts(normalize=True). \
#                 reset_index(). \
#                 rename(columns={'index': i, i: i + '_PART_TARGET=0'})
#             test_ = test_.fillna('NO_DATA', inplace=False).merge(tmp, how='left')
#             test_[i + '_PART_TARGET=0'].fillna(0, inplace=True)
#         return train_.append(test_).drop(list(self.categorial_columns()) + ['TARGET'] +
#                                          list(self.numeric_columns()), axis=1)

#     def getdummies(self, part=0.1):
#         #         Работает
#         train_ = self.train.copy()
#         test_ = self.test.copy()
#         categorial_column_unique_values = self.categorial_column_unique_values(part)
#         columns = list()
#         for i in self.categorial_columns():
#             for j in range(len(categorial_column_unique_values[i])):
#                 columns.append(str(i + '=' + str(categorial_column_unique_values[i][j])))
#                 train_[i + '=' + str(categorial_column_unique_values[i][j])] = \
#                     train_[i].apply(lambda x: 1 if x == str(categorial_column_unique_values[i][j]) else 0)
#         for i in self.categorial_columns():
#             for j in range(len(categorial_column_unique_values[i])):
#                 test_[i + '=' + str(categorial_column_unique_values[i][j])] = \
#                     test_[i].apply(lambda x: 1 if x == str(categorial_column_unique_values[i][j]) else 0)
#         return train_.append(test_)[columns]